In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

2024-09-11 13:35:28.110237: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
import tensorflow as tf

# Check if GPU is available_
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))



Num GPUs Available:  2


2024-09-11 13:35:31.760598: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-09-11 13:35:31.762068: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2024-09-11 13:35:32.086458: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:17:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.57GiB deviceMemoryBandwidth: 573.69GiB/s
2024-09-11 13:35:32.089699: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:65:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.56GiB deviceMemoryBandwidth: 573.69GiB/s
2024-09-11 13:35:32.089727: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart

In [24]:
# CSV-Daten laden (ersetze 'your_data_file.csv' mit dem tatsächlichen Dateinamen)
df = pd.read_csv('csv/processed_Trajectories.csv')

In [25]:
# Extrahiere die relevanten Daten (Schulter, Ellbogen, Handgelenk, ThumbTip)
shoulder = df[['Tasch:shoulder:X (mm)', 'Tasch:shoulder:Y (mm)', 'Tasch:shoulder:Z (mm)']].values
elbow = df[['Tasch:elbow:X (mm)', 'Tasch:elbow:Y (mm)', 'Tasch:elbow:Z (mm)']].values
wrist = df[['Tasch:wrist:X (mm)', 'Tasch:wrist:Y (mm)', 'Tasch:wrist:Z (mm)']].values
thumb_tip = df[['Tasch:ThumbTip:X (mm)', 'Tasch:ThumbTip:Y (mm)', 'Tasch:ThumbTip:Z (mm)']].values


In [26]:
# Normalisierung der Daten
scaler = MinMaxScaler()

X_all = np.vstack([shoulder, elbow, wrist])
X_all_scaled = scaler.fit_transform(X_all)

# Split the scaled data back into shoulder, elbow, and wrist
shoulder = X_all_scaled[:len(shoulder)]
elbow = X_all_scaled[len(shoulder):len(shoulder) + len(elbow)]
wrist = X_all_scaled[len(shoulder) + len(elbow):]

# Scale the thumb_tip data separately
thumb_tip = scaler.fit_transform(thumb_tip)

In [27]:
# Combine shoulder, elbow, wrist into the features (X)
X_train = np.hstack([shoulder, elbow, wrist])

# The target variable is thumb_tip (Y)
Y_train = thumb_tip

# Convert your NumPy arrays to TensorFlow datasets
dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))

# Set the dataset options and auto-shard policy to DATA
options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.DATA
dataset = dataset.with_options(options)

In [28]:
dataset = dataset.batch(32).shuffle(buffer_size=len(X_train)).prefetch(tf.data.AUTOTUNE)

In [29]:
# Define the strategy for using multiple GPUs
strategy = tf.distribute.MirroredStrategy()

print('Number of devices: {}'.format(strategy.num_replicas_in_sync))  # Should print 2 if 2 GPUs are detected

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Number of devices: 2


In [30]:
with strategy.scope():
    # Define the model inside the strategy scope
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(64, input_dim=X_train.shape[1], activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(3)  # Predicting X, Y, Z position of ThumbTip
    ])
    
    # Compile the model
    model.compile(
        optimizer='adam',                 # Adam optimizer
        loss='mean_squared_error',         # Mean squared error for regression
        metrics=['mean_absolute_error']    # Optional metric for monitoring
    )

In [31]:
# Modell trainieren
history_gpu = model.fit(X_train, Y_train, epochs=100, batch_size=32, validation_split=0.2)

# Visualisiere die Verlustkurve
plt.plot(history_gpu.history['loss'], label='Train Loss')
plt.plot(history_gpu.history['val_loss'], label='Validation Loss')
plt.title('Verlustkurve während des Trainings')
plt.xlabel('Epochen')
plt.ylabel('Verlust')
plt.legend()
plt.show()

2024-09-11 13:53:47.456270: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:656] In AUTO-mode, and switching to DATA-based sharding, instead of FILE-based sharding as we cannot find appropriate reader dataset op(s) to shard. Error: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_607820"
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset. You can do this by creating a new `tf.data.Options()` object then setting `options.experimental_distribute.a

Epoch 1/100
INFO:tensorflow:batch_all_reduce: 6 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 6 all-reduces with algorithm = nccl, num_packs = 1
1191/1198 [============================>.] - ETA: 0s - loss: 0.0217 - mean_absolute_error: 0.0750

2024-09-11 13:53:54.024015: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:656] In AUTO-mode, and switching to DATA-based sharding, instead of FILE-based sharding as we cannot find appropriate reader dataset op(s) to shard. Error: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_611802"
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset. You can do this by creating a new `tf.data.Options()` object then setting `options.experimental_distribute.a

1198/1198 [==============================] - 9s 6ms/step - loss: 0.0216 - mean_absolute_error: 0.0748 - val_loss: 0.0020 - val_mean_absolute_error: 0.0313
Epoch 2/100
1198/1198 [==============================] - 6s 5ms/step - loss: 9.5259e-04 - mean_absolute_error: 0.0208 - val_loss: 0.0013 - val_mean_absolute_error: 0.0251
Epoch 3/100
1198/1198 [==============================] - 6s 5ms/step - loss: 8.0839e-04 - mean_absolute_error: 0.0190 - val_loss: 0.0013 - val_mean_absolute_error: 0.0251
Epoch 4/100
1198/1198 [==============================] - 6s 5ms/step - loss: 7.5976e-04 - mean_absolute_error: 0.0184 - val_loss: 0.0013 - val_mean_absolute_error: 0.0236
Epoch 5/100
1198/1198 [==============================] - 6s 5ms/step - loss: 7.0989e-04 - mean_absolute_error: 0.0177 - val_loss: 0.0012 - val_mean_absolute_error: 0.0218
Epoch 6/100
1198/1198 [==============================] - 6s 5ms/step - loss: 7.1466e-04 - mean_absolute_error: 0.0177 - val_loss: 0.0018 - val_mean_absolute_erro

KeyboardInterrupt: 

In [ ]:
# Beispielhafte Vorhersage mit neuen Eingaben (simulierte Daten, ersetze durch tatsächliche Werte)
new_shoulder_position = np.array([[0.2, 0.3, 0.4]])  # Normalisierte Eingabe
new_elbow_position = np.array([[0.3, 0.4, 0.5]])     # Normalisierte Eingabe
new_wrist_position = np.array([[0.4, 0.5, 0.6]])     # Normalisierte Eingabe

In [ ]:
# Kombiniere die neuen Eingaben
new_input = np.hstack([new_shoulder_position, new_elbow_position, new_wrist_position])

# Vorhersage der ThumbTip Position
predicted_thumb_tip = model.predict(new_input)
print(f"Vorhergesagte ThumbTip Position: {predicted_thumb_tip}")
